In [2]:
import pandas as pd

jedi= pd.read_csv('csv_datoteke/jedi.csv',delimiter = ";")
lokali = pd.read_csv('csv_datoteke/lokali.csv',delimiter = ";")
jedi

,id_lokala,jed,vrsta
0,3186,PEČEN KROMPIR,Vegetarijansko
1,3186,PREBRANAC,Vegetarijansko
2,3186,VEGIBAB,Hitra hrana
3,3186,FALAFEL,Hitra hrana
4,3186,MEHIŠKA TORTILJA,Hitra hrana
...,...,...,...
3284,3077,RIŽEVI REZANCI S ŠKAMPI,Celiakiji prijazni obroki
3285,3077,REZANCI Z ZELENJAVO,Mešano
3286,3077,RIŽ Z ZELENJAVO,Celiakiji prijazni obroki
3287,3077,RIŽEVI REZANCI Z ZELENJAVO,Celiakiji prijazni obroki


In [10]:
verjetnosti_vrst = jedi.groupby('vrsta').count()['jed'] / len(jedi)
verjetnosti_vrst

vrsta
Celiakiji prijazni obroki    0.017635
Hitra hrana                  0.105807
Meso                         0.286409
Mešano                       0.127394
Pizza                        0.185771
Riba                         0.091517
Solata                       0.081180
Vegetarijansko               0.103375
Name: jed, dtype: float64

In [34]:
def koren_besede(beseda):
    beseda = ''.join(znak for znak in beseda if znak.isalpha())
    if not beseda:
        return '$'
    brez_koncnice = beseda.rstrip('ds')
    koren = brez_koncnice.rstrip('aeiou')
    if koren:
        return koren
    else:
        return beseda

def koreni_besed(niz):
    return pd.Series(sorted({
        koren_besede(beseda) for beseda in niz.replace('-', ' ').lower().split() if beseda
    }))

koreni_jedi = jedi.jed.apply(koreni_besed)
koreni_vrst = pd.merge(
    koreni_jedi.stack().reset_index()[['level_0', 0]],
    jedi,
    left_on='level_0',
    right_index=True
).rename(columns={0: 'koren'})[['koren', 'vrsta']]
pojavitve_korenov_po_vrstah = koreni_vrst.groupby(['koren', 'vrsta']).size().unstack()
pojavitve_korenov_po_vrstah


vrsta,Celiakiji prijazni obroki,Hitra hrana,Meso,Mešano,Pizza,Riba,Solata,Vegetarijansko
koren,,,,,,,,
$,NaN,38.0,70.0,13.0,94.0,15.0,14.0,16.0
a,NaN,NaN,2.0,3.0,NaN,1.0,NaN,NaN
aertičok,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
agl,NaN,NaN,NaN,3.0,NaN,NaN,NaN,2.0
aid,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
žepek,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0
žepk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
žit,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN


In [36]:

verjetnosti_korenov_po_vrstah = (pojavitve_korenov_po_vrstah / jedi.groupby('vrsta').size()).fillna(0.0001)
verjetnosti_korenov_po_vrstah


vrsta,Celiakiji prijazni obroki,Hitra hrana,Meso,Mešano,Pizza,Riba,Solata,Vegetarijansko
koren,,,,,,,,
$,0.0001,0.109195,0.074310,0.031026,0.153846,0.049834,0.052434,0.047059
a,0.0001,0.000100,0.002123,0.007160,0.000100,0.003322,0.000100,0.000100
aertičok,0.0001,0.000100,0.000100,0.000100,0.003273,0.000100,0.000100,0.000100
agl,0.0001,0.000100,0.000100,0.007160,0.000100,0.000100,0.000100,0.005882
aid,0.0001,0.000100,0.000100,0.000100,0.001637,0.000100,0.000100,0.000100
...,...,...,...,...,...,...,...,...
žepek,0.0001,0.000100,0.000100,0.000100,0.001637,0.000100,0.000100,0.002941
žepk,0.0001,0.000100,0.000100,0.000100,0.000100,0.000100,0.000100,0.005882
žit,0.0001,0.000100,0.000100,0.000100,0.001637,0.000100,0.003745,0.000100


In [39]:
hrana = "enolončnica"
koreni_hrana = koreni_besed(hrana)

stevec_vrjetnosti = (verjetnosti_korenov_po_vrstah [verjetnosti_korenov_po_vrstah .index.isin(koreni_hrana)].prod() * verjetnosti_vrst)
stevec_vrjetnosti.sort_values(ascending=False)

vrsta
Mešano                       0.002736
Celiakiji prijazni obroki    0.000304
Meso                         0.000029
Pizza                        0.000019
Hitra hrana                  0.000011
Vegetarijansko               0.000010
Riba                         0.000009
Solata                       0.000008
dtype: float64